<a href="https://colab.research.google.com/github/jaarhday/FolktaleTextGenerator/blob/main/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

## I. Parse Text Sources
First we'll load our text sources and create our vocabulary lists and encoders.

In [2]:
# Load file data
path_to_file = tf.keras.utils.get_file('tales.txt', 'https://raw.githubusercontent.com/jaarhday/FolktaleTextGenerator/main/complete_data.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1917369 characters


In [3]:
# Verify the first part of our data
print(text[:200])

Preface

In the beginning of the New-making, the ancient fathers lived
successively in four caves in the Four fold-containing-earth. The first
was of sooty blackness, black as a chimney at night time;


In [4]:
# Now we'll get a list of the unique characters in the file. This will form the
# vocabulary of our network. There may be some characters we want to remove from this
# set as we refine the network.
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

82 unique characters
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ã', 'ä', 'é']


In [5]:
# Next, we'll encode encode these characters into numbers so we can use them
# with our neural network, then we'll create some mappings between the characters
# and their numeric representations
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

# Here's a little helper function that we can use to turn a sequence of ids
# back into a string:
# turn them into a string:
def text_from_ids(ids):
  joinedTensor = tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
  return joinedTensor.numpy().decode("utf-8")

In [6]:
# Now we'll verify that they work, by getting the code for "A", and then looking
# that up in reverse
testids = ids_from_chars(["T", "r", "u", "t", "h"])
testids

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([44, 71, 74, 73, 61])>

In [7]:
chars_from_ids(testids)

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'T', b'r', b'u', b't', b'h'], dtype=object)>

In [8]:
testString = text_from_ids( testids )
testString

'Truth'

## II. Construct our training data
Next we need to construct our training data by building sentence chunks. Each chunk will consist of a sequence of characters and a corresponding "next sequence" of the same length showing what would happen if we move forward in the text. This "next sequence" becomes our target variable.



In [9]:
# First, create a stream of encoded integers from our text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1917369,), dtype=int64, numpy=array([40, 71, 58, ..., 75, 58,  3])>

In [10]:
# Now, convert that into a tensorflow dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [11]:
# Finally, let's batch these sequences up into chunks for our training
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Call the function for every sequence in our list to create a new dataset
# of input->target pairs
dataset = sequences.map(split_input_target)

In [12]:
# Verify our sequences
for input_example, target_example in  dataset.take(1):
    print("Input: ", text_from_ids(input_example))
    print("--------")
    print("Target: ", text_from_ids(target_example))

Input:  Preface

In the beginning of the New-making, the ancient fathers lived
successively in four caves in
--------
Target:  reface

In the beginning of the New-making, the ancient fathers lived
successively in four caves in 


In [13]:
# Finally, we'll randomize the sequences so that we don't just memorize the books
# in the order they were written, then build a new streaming dataset from that.
# Using a streaming dataset allows us to pass the data to our network bit by bit,
# rather than keeping it all in memory. We'll set it to figure out how much data
# to prefetch in the background.

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## III. Build the model

Next, we'll build our model.


In [14]:
class TextModel(tf.keras.Model):

  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)

    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    x = self.embedding(x, training=training)

    if states is None:
      states = self.gru.get_initial_state(x)

    x, states = self.gru(x, initial_state=states, training=training)

    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [15]:
# Create an instance of our model
vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

model = TextModel(vocab_size, embedding_dim, rnn_units)

In [16]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 83) # (batch_size, sequence_length, vocab_size)


In [17]:
# Now let's view the model summary
model.summary()

Model: "text_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  21248     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  85075     
                                                                 
Total params: 4044627 (15.43 MB)
Trainable params: 4044627 (15.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## IV. Train the model

For our purposes, we'll be using [categorical cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/) as our loss function*. Also, our model will be outputting ["logits" rather than normalized probabilities](https://stackoverflow.com/questions/41455101/what-is-the-meaning-of-the-word-logits-in-tensorflow), because we'll be doing further transformations on the output later.

In [18]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(dataset, epochs=25)

Epoch 1/25
296/296 [==============================] - 23s 57ms/step - loss: 2.3712
Epoch 2/25
296/296 [==============================] - 18s 55ms/step - loss: 1.7302
Epoch 3/25
296/296 [==============================] - 19s 56ms/step - loss: 1.5108
Epoch 4/25
296/296 [==============================] - 19s 58ms/step - loss: 1.3984
Epoch 5/25
296/296 [==============================] - 19s 56ms/step - loss: 1.3293
Epoch 6/25
296/296 [==============================] - 18s 56ms/step - loss: 1.2781
Epoch 7/25
296/296 [==============================] - 18s 55ms/step - loss: 1.2341
Epoch 8/25
296/296 [==============================] - 18s 56ms/step - loss: 1.1939
Epoch 9/25
296/296 [==============================] - 18s 56ms/step - loss: 1.1553
Epoch 10/25
296/296 [==============================] - 18s 56ms/step - loss: 1.1170
Epoch 11/25
296/296 [==============================] - 19s 57ms/step - loss: 1.0784
Epoch 12/25
296/296 [==============================] - 21s 59ms/step - loss: 1.0389
E

## V. Use the model

Now that our model has been trained, we can use it to generate text. As mentioned earlier, to do so we have to keep track of its internal state, or memory, so that we can use previous text predictions to inform later ones.

In [19]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask,validate_indices=False)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states =  self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Return the characters and model state.
    return chars_from_ids(predicted_ids), states


In [20]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

states = None

# Set what we want the beginning prompt to be, and let the model finish the rest!
next_char = tf.constant(['The world seemed like such a peaceful place until the magic tree was discovered in London.'])
result = [next_char]

for n in range(5000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

The world seemed like such a peaceful place until the magic tree was discovered in London.'

"'Alford I have nuts the earth on I'L Duv Lana'n, a snake's horse, I have searce!"

Wenonah, and such a point should be characters, and when ruin about
the budges of brave joined claws, a soldier, velvious and person was
then absent.

That morning Gwynner, Apara snakes here in his ears and ducked it, and when she
looked from its hunting cloak toward Puck to fight, and when he looked again
looking up the mast extended on the stones and casts him down, for the
populace of the people came to him in a vain.
He gazed on his hands to feed through the shade, and rolled his
hands beneath this dress, ridden flushed, steep cannoes from the
water, and up in the dark, though he was going on a bridle until the
pebble we had ever happened that he was affianced to them. But bluffy she pauses
never that Little Daughter was the only light, he followed the streams at
a city and near the others.

"Fionn saw you t